# 导包，加载图片，组成4D的张量

- 图片放在`data`目录下，使用`img_to_array()`加载图片为numpy arrays
- 将图片按batch组合，同时处理多张图片，扩充程序的扩展性。

In [1]:
import numpy as np
from PIL import Image
from utils import  img_to_array,array_to_img,visualize_grid,view_images

# params
DIMS = (400, 400)
CAT1 = 'cat1.jpg'
CAT2 = 'cat2.jpg'
data_path = './data/'

# 加载两张小猫图片
img1 = img_to_array(data_path + CAT1, DIMS)
img2 = img_to_array(data_path + CAT2, DIMS, view=True)

# 联合两张图片到一个batch,shape为(2, 400, 400, 3)
input_img = np.concatenate([img1, img2], axis=0)

print(img2.shape)
print("Input Img Shape: {}".format(input_img.shape))

(1, 400, 400, 3)
Input Img Shape: (2, 400, 400, 3)


# 构建仿射变换生成矩阵和双线性插值函数

`affine_grid_generator(height, width, M)`为仿射变换函数:

- height,width,为图片长和宽
- M： 仿射矩阵。 shape为(num_batch, 2, 3). 
- Return: 返回sampling grid。shape为(num_batch, H, W, 2)

`bilinear_sampler(input_img, x, y)`为双线性采样：

- input_img：采样的原始图片，(B, H, W, C)
- x,y: affine_grid_generator输出
- Return：采样后的值

In [2]:
def affine_grid_generator(height, width, M):
    num_batch = M.shape[0] # 获取batchsize

    # 创建棋盘grid,平分整个图片
    x = np.linspace(-1, 1, width)
    y = np.linspace(-1, 1, height)
    x_t, y_t = np.meshgrid(x, y)

    # 制作列向量(xt, yt, 1) 
    ones = np.ones(np.prod(x_t.shape))
    sampling_grid = np.vstack([x_t.flatten(), y_t.flatten(), ones])
    # 把所有像素整到一起  sampling_grid的shape为(batch, 3, H*W)  
    sampling_grid = np.resize(sampling_grid, (num_batch, 3, height*width)) # 列向量扩展batch
    
    # 做仿射矩阵运算 M*K
    batch_grids = np.matmul(M, sampling_grid)
    # batch grid 的 shape (num_batch, 2, H*W)
    
    # 把H,W拆开
    batch_grids = batch_grids.reshape(num_batch, 2, height, width)
    batch_grids = np.moveaxis(batch_grids, 1, -1) #调整为(num_batch, H, W, 2)

    # sanity check
    print("Transformation Matrices: {}".format(M.shape))
    print("Sampling Grid: {}".format(sampling_grid.shape))
    print("Batch Grids: {}".format(batch_grids.shape))

    return batch_grids


def bilinear_sampler(input_img, x, y):
    """
    如果想要测试是否正常，将仿射变换改为恒等变换，查看是输入和输出
    """
    # grab dimensions
    B, H, W, C = input_img.shape

    # 原本x,y在[-1,1]之间，现在放缩到 [0, W/H]
    x = ((x + 1.) * W) * 0.5
    y = ((y + 1.) * H) * 0.5

    # 获取到每个(x_i, y_i)相邻的4个坐标
    x0 = np.floor(x).astype(np.int64)
    x1 = x0 + 1
    y0 = np.floor(y).astype(np.int64)
    y1 = y0 + 1

    # 确保不会超界  make sure it's inside img range [0, H] or [0, W]
    x0 = np.clip(x0, 0, W-1)
    x1 = np.clip(x1, 0, W-1)
    y0 = np.clip(y0, 0, H-1)
    y1 = np.clip(y1, 0, H-1)

    # 取出对应的像素值
    Ia = input_img[np.arange(B)[:,None,None], y0, x0]
    Ib = input_img[np.arange(B)[:,None,None], y1, x0]
    Ic = input_img[np.arange(B)[:,None,None], y0, x1]
    Id = input_img[np.arange(B)[:,None,None], y1, x1]

    # 依据双线性插值公式，计算deltas
    wa = (x1-x) * (y1-y)
    wb = (x1-x) * (y-y0)
    wc = (x-x0) * (y1-y)
    wd = (x-x0) * (y-y0)

    # add dimension for addition
    wa = np.expand_dims(wa, axis=3)
    wb = np.expand_dims(wb, axis=3)
    wc = np.expand_dims(wc, axis=3)
    wd = np.expand_dims(wd, axis=3)

    # compute output
    out = wa*Ia + wb*Ib + wc*Ic + wd*Id

    return out


## 设置仿射矩阵

- 设置仿射矩阵的值，对应不同的仿射变换
- 调节仿射矩阵的batch维度

In [3]:
B, H, W, C = input_img.shape

# 设置仿射矩阵
#M = np.array([[1., 0., 0.], [0., 1., 0.]]) 恒等变换
M = np.array([[1., 0., 0.5], [0., 1., 0.]]) #平移
#M = np.array([[0.707, -0.707, 0.], [0.707, 0.707, 0.]]) #旋转

# 调整维度
M = np.resize(M, (B, 2, 3))
print (M.shape)

(2, 2, 3)


## 获取仿射矩阵采样点，采样

- 获取仿射矩阵生成的采样点
- 将x,y维度拆分
- 使用双线性插值采样

In [4]:
batch_grids = affine_grid_generator(H, W, M)

x_s = batch_grids[:, :, :, 0:1].squeeze()
y_s = batch_grids[:, :, :, 1:2].squeeze()

out = bilinear_sampler(input_img, x_s, y_s)

Transformation Matrices: (2, 2, 3)
Sampling Grid: (2, 3, 160000)
Batch Grids: (2, 400, 400, 2)


## 查看变换图片

In [5]:
out = array_to_img(out[-1])
out.show()